# Exercise 3

I had some trouble with this excercise. As I'm using SQLITE as the DDBB Manager, I have some limitations. SQLITE can support JSON strings but it doesn't handle list variables, so it is not possible make a query that satisfys specifically the requirements. Nevertheless, I have created a similar DDBB as the one described in the intructions and I have added an extra table that contains the same information that is in the third JSON atribute. 

In [1]:
import sqlite3 as lite
import random
import pandas as pd
import json

In [2]:
BBDD = 'GlovoAppEx3.db'

In [3]:
database = lite.connect(BBDD)
c = database.cursor()

try:
    c.execute('''
    DROP TABLE Events
    ''')
except:
    pass


try:
    c.execute('''
    DROP TABLE StoreId
    ''')
except:
    pass

query_create_events = '''
CREATE TABLE "Events" (
	"id"	INTEGER NOT NULL,
	"Creation_Time"	DATETIME NOT NULL,
	"Attributes_Json"	TEXT NOT NULL,
	PRIMARY KEY("id")
)
'''

query_create_storeId = '''
CREATE TABLE "StoreId" (
	"id"	INTEGER NOT NULL,
    "Event_Id"	INTEGER NOT NULL,
	"Position"	INTEGER NOT NULL,
	"StoreId"	INTEGER NOT NULL,
	PRIMARY KEY("id")
)
'''

c.execute(query_create_events)

c.execute(query_create_storeId)

database.commit()


In [4]:
def insert_event(Event_Id, Creation_Time, Attributes_Json):
    database = lite.connect(BBDD)
    c = database.cursor()
        
    query_params = [Event_Id, Creation_Time, Attributes_Json]
    
    query_insert_event = '''
    INSERT INTO Events (Id, Creation_Time, Attributes_Json)
              VALUES(?,?,?)
    '''
    
    c.execute(query_insert_event, query_params)
    database.commit()
    
def insert_storeId(Event_Id, Position, StoreId):
    database = lite.connect(BBDD)
    c = database.cursor()
        
    query_params = [Event_Id, Position, StoreId]
    
    query_insert_event = '''
    INSERT INTO StoreId (Event_Id, Position, StoreId)
              VALUES(?,?,?)
    '''
    
    c.execute(query_insert_event, query_params)
    database.commit()

In [5]:
from random import randrange
from datetime import timedelta
from datetime import datetime


def random_date():
    start = datetime.strptime('1/1/2008 1:30 PM', '%m/%d/%Y %I:%M %p')
    end = datetime.strptime('4/30/2008 4:50 AM', '%m/%d/%Y %I:%M %p')
    
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [6]:
def create_event(Event_Id):
    
    Creation_Time = random_date()
    
    json_dictionary = {
        'country_code': 'ES',
        'city_code': 'BCN',
        'storeIds': ''
    }
    
    stores_list = random.sample(range(100),100)
    stores_str = str(stores_list)
    json_dictionary['storeIds'] = stores_list
    
    Attributes_Json = json.dumps(json_dictionary, indent = 3)
    
    insert_event(Event_Id,Creation_Time,Attributes_Json)
    
    for i in range(len(stores_list)):
        insert_storeId(Event_Id, i+1, stores_list[i])
        
    
    

In [7]:
for i in range(40):
    create_event(i)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


## This is the final Query



In [ ]:
SELECT 
		StoreId as Store, 
		sum(S.Position)/count(*) as Average_Position,
		strftime('%m',E.Creation_Time) as Month,
		strftime('%Y',E.Creation_Time) as Year,
		json_extract(E.Attributes_Json, '$.country_code') as Country_Code,
		json_extract(E.Attributes_Json, '$.city_code') as City_Code,
        count(*) as Events_in_Month

FROM Events E
LEFT JOIN StoreId S
on E.id = S.Event_Id 

GROUP by 
	strftime('%m',E.Creation_Time), 
	StoreId, 
	json_extract(E.Attributes_Json, '$.country_code'), 
	json_extract(E.Attributes_Json, '$.city_code')
	
order by StoreId, Year, Month

In [11]:
query_stores = '''
SELECT 
		StoreId as Store, 
		sum(S.Position)/count(*) as Average_Position,
		strftime('%m',E.Creation_Time) as Month,
		strftime('%Y',E.Creation_Time) as Year,
		json_extract(E.Attributes_Json, '$.country_code') as Country_Code,
		json_extract(E.Attributes_Json, '$.city_code') as City_Code,
        count(*) as Events_in_Month

FROM Events E
LEFT JOIN StoreId S
on E.id = S.Event_Id 

GROUP by 
	strftime('%m',E.Creation_Time), 
	StoreId, 
	json_extract(E.Attributes_Json, '$.country_code'), 
	json_extract(E.Attributes_Json, '$.city_code')
	
order by StoreId, Year, Month
'''

In [12]:
# Read sqlite query results into a pandas DataFrame
database = lite.connect(BBDD)
df = pd.read_sql_query(query_stores, database)

# Verify that result of SQL query is stored in the dataframe

database.close()

In [13]:
df

,Store,Average_Position,Month,Year,Country_Code,City_Code,Events_in_Month
0,0,37,01,2008,ES,BCN,11
1,0,54,02,2008,ES,BCN,9
2,0,41,03,2008,ES,BCN,12
3,0,61,04,2008,ES,BCN,8
4,1,41,01,2008,ES,BCN,11
...,...,...,...,...,...,...,...
395,98,47,04,2008,ES,BCN,8
396,99,60,01,2008,ES,BCN,11
397,99,55,02,2008,ES,BCN,9
398,99,47,03,2008,ES,BCN,12


This dataframe contains the averange position of each restaurant for every month, year, country and city, as demanded in the intructions.